# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [ ]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [ ]:
%pip install --upgrade plotly

### Import Statements

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [6]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`?
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [9]:
df_data.head()

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [ ]:
print(f"NaN values: {df_data.isnull().values.any()}")
print(f"Duplicate rows: {df_data.duplicated().any()}")


NaN values: True
Duplicate rows: False


## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data.

In [10]:
df_data.dropna(inplace=True)

## Descriptive Statistics

In [ ]:
df_data.describe()

,Unnamed: 0.1,Unnamed: 0
count,964.00,964.00
mean,858.49,858.49
std,784.21,784.21
min,0.00,0.00
25%,324.75,324.75
50%,660.50,660.50
75%,"1,112.00","1,112.00"
max,"4,020.00","4,020.00"


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [ ]:
launches_by_company = df_data.groupby('Organisation')['Organisation'].count()
launches_by_company.sort_values(ascending=False, inplace=True)
launches_by_company

In [ ]:
fig = px.bar(x=launches_by_company.index,
             y=launches_by_company.values,
             title="Launches Per Company",
             color=launches_by_company.index )
fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned?

In [12]:
active_vs_retired_rockets = df_data.groupby('Rocket_Status').agg({'Rocket_Status': 'count'})
active_vs_retired_rockets

,Rocket_Status
Rocket_Status,
StatusActive,586
StatusRetired,378


# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [13]:
successful_vs_failed_missions = df_data.groupby('Mission_Status').agg({'Mission_Status': 'count'})
successful_vs_failed_missions

,Mission_Status
Mission_Status,
Failure,36
Partial Failure,17
Prelaunch Failure,1
Success,910


# How Expensive are the Launches?

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values).

In [14]:
df_price = df_data.dropna(subset=['Price'])
price_graph = px.histogram(df_price,
                           x='Price',
                           nbins=20,
                           title='Distribution of Launch Prices (USD Millions)',
                           labels={'Price': 'Launch Price (USD Millions)'}
                          )
price_graph.show()


# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map.
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [20]:
df_data['Country'] = df_data['Location'].str.split(',').str[-1].str.strip()
df_data['Country'].unique()
df_data['Country']

replaced_countries = {
    'Russia': 'Russian Federation',
    'New Mexico': 'USA',
    'Yellow Sea': 'China',
    'Shahrud Missile Test Site': 'Iran',
    'Pacific Missile Range Facility': 'USA',
    'Barents Sea': 'Russian Federation',
    'Gran Canaria': 'USA'
}

df_data['Country'].replace(replaced_countries)


,Country
0,USA
1,China
3,Kazakhstan
4,USA
5,China
...,...
3855,USA
3971,USA
3993,USA
4000,USA


In [22]:
def get_country_code(country):
    try:
        return countries.get(country).alpha3
    except:
        return np.nan

df_data['Country_Code'] = df_data['Country'].apply(get_country_code)

launches_by_country = df_data.groupby('Country_Code')['Country_Code'].count()
launches_by_country.sort_values(ascending=False, inplace=True)
launches_by_country


,Country_Code
Country_Code,
USA,487
CHN,158
FRA,95
IND,67
KAZ,46
JPN,40
NZL,13


In [23]:
fig = px.choropleth(
    locations=launches_by_country.index,
    locationmode='ISO-3',
    color=launches_by_country.values,
    hover_name=launches_by_country.index,
    color_continuous_scale=px.colors.sequential.matter,
    title='Number of Launches by Country'
)
fig.show()


# Use a Choropleth Map to Show the Number of Failures by Country


In [24]:
failure_by_country = df_data[df_data['Mission_Status'] == 'Failure'].groupby('Country_Code')['Country_Code'].count()
failure_by_country.sort_values(ascending=False, inplace=True)
failure_by_country

,Country_Code
Country_Code,
USA,19
IND,5
CHN,4
FRA,2
KAZ,2
NZL,2


In [25]:
failure_by_country_graph = px.choropleth(
    locations=failure_by_country.index,
    locationmode='ISO-3',
    color=failure_by_country.values)

failure_by_country_graph.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status.

In [27]:
sunburst_chart = px.sunburst(df_data,
                             path=['Country', 'Organisation', 'Mission_Status'],
                             title='Mission Status by Country and Organisation')
sunburst_chart.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [28]:
df_data["Price"] = pd.to_numeric(df_data["Price"], errors='coerce')
money_spent_by_organisation = df_data.groupby('Organisation').agg({"Price":"sum"})
money_spent_by_organisation.sort_values(by="Price", ascending=False, inplace=True)
money_spent_by_organisation

,Price
Organisation,
NASA,"61,200.00"
Arianespace,"16,345.00"
ULA,"14,798.00"
CASC,"6,340.26"
SpaceX,"5,444.00"
Northrop,"3,930.00"
MHI,"3,532.50"
ISRO,"2,177.00"
US Air Force,"1,550.92"


# Analyse the Amount of Money Spent by Organisation per Launch

In [29]:
average_spent_per_launch = df_data.groupby('Organisation').agg({"Price": "mean"})
average_spent_per_launch.sort_values(by="Price", ascending=False, inplace=True)
average_spent_per_launch

fig = px.bar(
    average_spent_per_launch,
    x=average_spent_per_launch.index,
    y="Price",
    color=average_spent_per_launch.index,
    title="Average Amount Spent per Launch by Organisation",
    labels={"Price": "Average Price (USD Millions)"},
)
fig.show()


# Chart the Number of Launches per Year

In [30]:
df_data["Date"] = pd.to_datetime(df_data['Date'], format='%a %b %d, %Y %H:%M %Z', utc=True, errors='coerce')
df_data["Year"] = df_data["Date"].dt.year.astype("Int64")
df_data["Year"]

,Year
0,2020
1,2020
3,2020
4,2020
5,2020
...,...
3855,1966
3971,1965
3993,1965
4000,1964


In [31]:
launches_per_year = df_data.groupby("Year").agg({"Year": "count"})
launches_per_year

,Year
Year,
1964,2
1965,2
1966,3
1967,7
1968,10
1969,8
1970,1
1971,2
1972,2


In [32]:
fig=px.bar(launches_per_year,
           x=launches_per_year.index,
           y="Year",
           color="Year")
fig.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart.

In [33]:
df_data["Month"] = df_data["Date"].dt.month.astype("Int64")
launches_by_month = df_data.groupby("Month").agg({"Month": "count"})
launches_by_month


,Month
Month,
1,66
2,60
3,77
4,88
5,86
6,85
7,77
8,82
9,87


In [34]:
fig = px.bar(launches_by_month,
                   x=launches_by_month.index,
                   y="Month",
                   color="Month",
                   title="Number of Launches per Month",
                   labels={"Month": "Number of Launches"})
fig.show()

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [35]:
best_launch_month = launches_by_month["Month"].idxmax()

print(f"The month with the highest number of launches is: {best_launch_month}")



The month with the highest number of launches is: 12


# How has the Launch Price varied Over Time?

Create a line chart that shows the average price of rocket launches over time.

In [36]:
price_over_time = df_data.groupby("Year").agg({"Price": "mean"})
price_over_time = price_over_time.dropna()
avg_price_over_time = price_over_time.rolling(window=5).mean()
avg_price_over_time

,Price
Year,
1964,NaN
1965,NaN
1966,NaN
1967,NaN
1968,60.69
1969,59.85
1981,137.20
1982,194.44
1983,255.87


In [37]:
fig = px.line(avg_price_over_time,
              x=avg_price_over_time.index,
              y="Price",
              title="Average Price of Rocket Launches Over Time",
              labels={"Price": "Average Price (USD Millions)"})
fig.show()

# Chart the Number of Launches over Time by the Top 10 Organisations.

How has the dominance of launches changed over time between the different players?

In [38]:
top_10_organisations = launches_by_company.head(10)

fig = px.bar(top_10_organisations,
             x=top_10_organisations.index,
             y="Organisation",
             color=top_10_organisations.index)
fig.show()




# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991.

In [51]:
df_russia = df_data[df_data['Country'] == 'Russia']
df_usa = df_data[df_data['Country'] == 'USA']
df_usa_vs_russia = pd.concat([df_russia, df_usa])
df_usa_vs_russia



,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Country,Country_Code,Year,Month
46,46,46,VKS RF,"Site 43/3, Plesetsk Cosmodrome, Russia",2020-02-20 08:24:00+00:00,Soyuz 2.1a/Fregat-M | Meridian-M n†­19L,StatusActive,48.50,Success,Russia,NaN,2020,2
64,64,64,VKS RF,"Site 133/3, Plesetsk Cosmodrome, Russia",2019-12-26 23:11:00+00:00,"Rokot/Briz KM | Gonets-M ???24, 25, 26 [block-...",StatusRetired,41.80,Success,Russia,NaN,2019,12
73,73,73,VKS RF,"Site 43/3, Plesetsk Cosmodrome, Russia",2019-12-11 08:54:00+00:00,Soyuz 2.1b/Fregat | Cosmos 2544,StatusActive,48.50,Success,Russia,NaN,2019,12
97,97,97,VKS RF,"Site 43/4, Plesetsk Cosmodrome, Russia",2019-09-26 07:46:00+00:00,Soyuz 2.1b/Fregat | Cosmos 2541,StatusActive,48.50,Success,Russia,NaN,2019,9
105,105,105,VKS RF,"Site 133/3, Plesetsk Cosmodrome, Russia",2019-08-30 14:00:00+00:00,Rokot/Briz KM | Cosmos 2540,StatusRetired,41.80,Success,Russia,NaN,2019,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3855,3855,3855,US Air Force,"SLC-4W, Vandenberg AFB, California, USA",1966-07-29 18:43:00+00:00,Titan IIIB | KH-8,StatusRetired,59.00,Success,USA,USA,1966,7
3971,3971,3971,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA",1965-05-06 15:00:00+00:00,Titan IIIA | LES 2 & LCS 1,StatusRetired,63.23,Success,USA,USA,1965,5
3993,3993,3993,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA",1965-02-11 15:19:00+00:00,Titan IIIA | LES 1,StatusRetired,63.23,Success,USA,USA,1965,2
4000,4000,4000,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA",1964-12-10 16:52:00+00:00,Titan IIIA | Transtage 2,StatusRetired,63.23,Success,USA,USA,1964,12


## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches.

In [52]:
launches_by_country_usa_ussr = df_usa_vs_russia.groupby('Country')['Country'].count()

fig = px.pie(
    values=launches_by_country_usa_ussr.values,
    names=launches_by_country_usa_ussr.index,
    title="Launches by Country: USA vs. USSR/Russia",
)
fig.show()


## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [53]:
launches_by_year_country = df_usa_vs_russia.groupby(['Year', 'Country'])['Country'].count().unstack()

fig = px.line(
    launches_by_year_country,
    x=launches_by_year_country.index,
    y=['USA', 'Russia'],
    title='Year-on-Year Launches: USA vs. Russia/USSR',
    labels={'value': 'Number of Launches', 'variable': 'Country'},
)
fig.show()


## Chart the Total Number of Mission Failures Year on Year.

In [57]:
mission_failures_by_year = df_data[df_data['Mission_Status'] == 'Failure'].groupby('Year')['Year'].count()
# print(mission_failures_by_year)

fig = px.bar(
    mission_failures_by_year,
    x=mission_failures_by_year.index,
    y='Year',
    title='Total Number of Mission Failures Year on Year',
    labels={'Year': 'Number of Failures'},
)
fig.show()


## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time?

In [59]:
df_data['Year'] = pd.to_datetime(df_data['Date']).dt.year
total_launches_by_year = df_data.groupby('Year')['Mission_Status'].count()
failures_by_year = df_data[df_data['Mission_Status'] == 'Failure'].groupby('Year')['Mission_Status'].count()
failure_percentage_by_year = (failures_by_year / total_launches_by_year * 100).fillna(0)

fig = px.line(
    failure_percentage_by_year,
    x=failure_percentage_by_year.index,
    y=failure_percentage_by_year.values,
    title='Percentage of Mission Failures Over Time',
    labels={'x': 'Year', 'y': 'Percentage of Failures'},
)
fig.show()


# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches?

In [66]:
successful_launches_by_year_country = df_data[df_data['Mission_Status'] == 'Success'].groupby(['Year', 'Country'])['Country'].count().unstack()

leading_country_successful = successful_launches_by_year_country.idxmax(axis=1)

print("Leading Country Based on Successful Launches (up to 2020):")
leading_country_successful


Leading Country Based on Successful Launches (up to 2020):


,0
Year,
"1,964.00",USA
"1,965.00",USA
"1,966.00",USA
"1,967.00",USA
"1,968.00",USA
"1,969.00",USA
"1,970.00",USA
"1,971.00",USA
"1,972.00",USA


# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020?

In [68]:
launches_by_year_org = df_data.groupby(['Year', 'Organisation'])['Organisation'].count().unstack()
dominant_org_by_year = launches_by_year_org.idxmax(axis=1)

fig = px.bar(
    launches_by_year_org,
    x=dominant_org_by_year.index,
    y=dominant_org_by_year.values,
    title='Dominant Organisation in Space Launches Year-on-Year',
    labels={'value': 'Dominant Organisation', 'x': 'Year'},
)
fig.show()
